In [2]:
import json
import boto3
import asyncio
import argparse
from boto3.session import Session

In [3]:
async def invoke_agent(question):
    boto_session = Session()
    region_name = boto3.Session().region_name


    #SSM을 활용해서 Bedrock AgenticCore ARN 가져오기
    ssm_client = boto_session.client('ssm', region_name=region_name)
    agent_arn_response = ssm_client.get_parameter(
        Name="/basic_server/runtime_iam/agent_arn"
    )
    agentcore_client = boto3.client('bedrock-agentcore', region_name=region_name)

    
    #Streaming으로 Bedrock AgenticCore Application 호출하기
    loop = asyncio.get_event_loop()
    response = await loop.run_in_executor(
        None,
        lambda: agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn_response["Parameter"]["Value"],
            qualifier="DEFAULT",
            payload=json.dumps({"input_data": question})
        )
    )

    #응답을 객체로 받은뒤 계속적으로 오는 streaming 결과 출력
    for line in response['response'].iter_lines():
        if line and line.startswith(b'data: '):
            data = json.loads(line[6:].decode('utf-8'))
            if data['type'] == 'stream':
                print(data['content'], end='')
    print()

In [4]:
await invoke_agent("langchain에 대해 설명해 주세요")


LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 오픈소스 프레임워크입니다.

## 주요 특징

### 1. **체인(Chain) 기반 구조**
- 여러 컴포넌트를 연결하여 복잡한 워크플로우 구성
- 단계별 처리를 통한 체계적인 작업 수행

### 2. **핵심 컴포넌트**

**프롬프트 템플릿 (Prompt Templates)**
```python
from langchain import PromptTemplate

template = "다음 질문에 {language}로 답해주세요: {question}"
prompt = PromptTemplate(
    input_variables=["language", "question"],
    template=template
)
```

**LLM 통합**
- OpenAI GPT, Anthropic Claude, Hugging Face 모델 등 지원
- 다양한 모델을 통일된 인터페이스로 사용

**메모리 관리**
- 대화 히스토리 저장 및 관리
- 컨텍스트 유지를 통한 연속적인 대화

### 3. **주요 기능들**

**문서 처리**
- PDF, 웹페이지, 텍스트 파일 등 다양한 형식 지원
- 문서 분할, 임베딩, 벡터 저장소 연동

**에이전트 (Agents)**
- 도구를 사용하여 자율적으로 작업 수행
- 웹 검색, 계산기, API 호출 등 외부 도구 활용

**RAG (Retrieval-Augmented Generation)**
- 외부 지식베이스와 연동
- 실시간 정보 검색 및 활용

## 사용 사례

1. **챗봇 개발**
2. **문서 요약 및 분석**
3. **질의응답 시스템**
4. **코드 생성 도구**
5. **데이터 분석 자동화**

## 장점

- **모듈화**: 재사용 가능한 컴포넌트
- **확장성**: 다양한 LLM과 도구 지원
- **커뮤니티**: 활발한 개발자 생태계
- **문서화**: 풍부한 예제와 가이드

LangChain은 LLM의 복잡한 기능들을 쉽게